## Setup

In [ ]:
!sudo apt-get install -y xvfb ffmpeg x11-utils
!pip install -q 'gym==0.10.11'
!pip install -q 'imageio==2.4.0'
!pip install -q PILLOW
!pip install -q 'pyglet==1.3.2'
!pip install -q pyvirtualdisplay
!pip install -q tf-agents
!pip install colabgymrender
!pip install gym_sokoban
!pip install gym
!apt-get install python-opengl -y
!apt-get install -y xvfb python-opengl > /dev/null 2>&1
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install python-opengl -y
!apt install xvfb -y
!pip install piglet

In [ ]:
import gym
import gym_sokoban

from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

from colabgymrender.recorder import Recorder
from IPython import display as ipythondisplay
from pyvirtualdisplay import Display
Display().start()
import matplotlib.pyplot as plt
from IPython import display as ipythondisplay


import gym
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T


is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

In [ ]:
env = gym.make("PushAndPull-Sokoban-v2")
env.max_steps =  200 
screen = env.render(mode='rgb_array')
print(screen.shape)

In [ ]:
plt.imshow(screen)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward', 'expected_reward'))


class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([],maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)
    
    def priority_sample(self, batch_size):
      diffs = np.array([np.power(np.absolute(x.reward.item() - x.expected_reward), 0.66)  for x in self.memory])
      s1 = np.sum(diffs)
      probs = diffs/s1

      inds = np.random.choice(len(self.memory), batch_size, replace=False)

      return [self.memory[i] for i in inds]




    def __len__(self):
        return len(self.memory)

In [ ]:
# class DQN(nn.Module):

#     def __init__(self, h, w, outputs):
#         super(DQN, self).__init__()
      

#         self.conv1 = nn.Conv2d(1, 32, kernel_size=8, stride=4)
#         #self.bn1 = nn.BatchNorm2d(16)
#         self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=2)
#         #self.bn2 = nn.BatchNorm2d(32)
#         self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1)
#         #self.bn3 = nn.BatchNorm2d(32)

#         self.pooling = nn.MaxPool2d(3, stride=2)

#         # Number of Linear input connections depends on output of conv2d layers
#         # and therefore the input image size, so compute it.
#         def conv2d_size_out(size, kernel_size = 5, stride = 2):
#             return (size - (kernel_size - 1) - 1) // stride  + 1

#         convw = conv2d_size_out(conv2d_size_out(conv2d_size_out(w)))
#         convh = conv2d_size_out(conv2d_size_out(conv2d_size_out(h)))
#         linear_input_size = convw * convh * 32

#         self.pre_adventage = nn.Linear(6400, 512)
#         self.adventage = nn.Linear(512, outputs)

#         self.pre_val = nn.Linear(6400, 512)
#         self.val = nn.Linear(512, 1)
 
#     # Called with either one element to determine next action, or a batch
#     # during optimization. Returns tensor([[left0exp,right0exp]...]).
#     def forward(self, x):
#         x = x.to(device)
#         # x = F.relu(self.bn1(self.conv1(x)))
#         # x = F.relu(self.bn2(self.conv2(x)))
#         # x = F.relu(self.bn3(self.conv3(x)))
#         x = F.relu(self.conv1(x))
#         x = F.relu(self.conv2(x))
#         x = F.relu(self.conv3(x))
#         x = x.view(x.size(0), -1)


       

#         value = F.relu(self.pre_val(x))
#         value =  self.val(value)

#         adv = F.relu(self.pre_adventage(x))

#         adv =  self.adventage(adv)

#         return value + adv - adv.mean()
  
#     def init_weights(self):
#       torch.nn.init.xavier_uniform(self.conv1.weight)
#       torch.nn.init.xavier_uniform(self.conv2.weight)
#       torch.nn.init.xavier_uniform(self.conv3.weight)
#       torch.nn.init.xavier_uniform(self.adventage.weight)
#       torch.nn.init.xavier_uniform(self.val.weight)
#       torch.nn.init.xavier_uniform(self.pre_adventage.weight)
#       torch.nn.init.xavier_uniform(self.pre_val.weight)


class DQN(nn.Module):
    def __init__(self, outputs):
        super().__init__()

        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)

        self.pre_adv = nn.Linear(10000, 512)
        self.adv = nn.Linear(512, outputs)

        self.pre_val = nn.Linear(10000, 512)
        self.val = nn.Linear(512, 1)
        

    def forward(self, x):
        x = x.to(device)
        x = self.pool1(F.leaky_relu(self.conv1(x)))

        
        x = self.pool2(F.leaky_relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch

        adv = F.leaky_relu(self.pre_adv(x))
        adv = self.adv(adv)

        value = F.leaky_relu(self.pre_val(x))
        value = self.val(value)
        
        
        return value + adv - adv.mean()

    
    def init_weights(self):
      torch.nn.init.xavier_uniform(self.conv1.weight)
      torch.nn.init.xavier_uniform(self.conv2.weight)
      
      torch.nn.init.xavier_uniform(self.pre_adv.weight)
      torch.nn.init.xavier_uniform(self.adv.weight)

      torch.nn.init.xavier_uniform(self.pre_val.weight)
      torch.nn.init.xavier_uniform(self.val.weight)

      return self


In [ ]:
# m1 = nn.MaxPool2d(3, stride=2)
# m2 = nn.MaxPool2d(3, stride=2)
# state = get_screen().unsqueeze(0)
# conv1 = nn.Conv2d(1, 32, kernel_size=8, stride=4)
# conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=2)
# conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1)

# i = conv1(state)
# x1 = i.shape

# i = m1(i)
# x2=i.shape
# i = conv2(i)
# i = m2(i)

# i.shape

# x = DQN(13)
# x = x.init_weights().to(device)




In [ ]:
resize = T.Compose([
                    T.ToPILImage(),
                    T.Grayscale(num_output_channels=1),
                    #T.Resize(80, interpolation=Image.CUBIC),
                    T.ToTensor()])

#resize = T.Compose([T.ToTensor()])


def get_screen():
    screen = env.render(mode='rgb_array')
    #return screen
    return resize(screen)


    

In [ ]:
BATCH_SIZE = 128
GAMMA = 0.9
EPS_START = 1.0
EPS_END = 0.1
EPS_DECAY = 60000
TARGET_UPDATE = 10

In [ ]:


# Get screen size so that we can initialize layers correctly based on shape
# returned from AI gym. Typical dimensions at this point are close to 3x40x90
# which is the result of a clamped and down-scaled render buffer in get_screen()
init_screen = get_screen()

_, screen_height, screen_width = init_screen.shape

# Get number of actions from gym action space
n_actions = env.action_space.n

policy_net = DQN(n_actions)
policy_net.init_weights()
policy_net.to(device)

target_net = DQN(n_actions).to(device)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

optimizer = optim.RMSprop(policy_net.parameters(), lr=0.001)
criterion = nn.SmoothL1Loss() #nn.SmoothL1Loss(reduction='mean', beta=0.3) #beta=0.1

# screen_height, screen_width

In [ ]:
def select_action(state, deterministic = False):
    global steps_done
    steps_done += 1

    # sample = random.random()
    # eps_threshold = EPS_END + (EPS_START - EPS_END) * \
    #     math.exp(-1. * steps_done / EPS_DECAY)
    
    # if sample > eps_threshold:
    #     with torch.no_grad():
    #         # t.max(1) will return largest column value of each row.
    #         # second column on max result is index of where max element was
    #         # found, so we pick action with the larger expected reward.
    #         return policy_net(state).max(1)[1].view(1, 1)
    # else:
    #     return torch.tensor([[random.randrange(n_actions)]], device=device, dtype=torch.long)

    with torch.no_grad():
      xxx = policy_net(state).cpu().detach().numpy().flatten()
     

      
      if deterministic:
        return xxx.max(1)[1].view(1, 1)
      else:
        m = xxx.min()
        vals = xxx - m + 0.0001 if m < 0 else xxx + 0.0001

        s  = np.sum(vals)
        ps = vals/s
        val = np.random.choice(n_actions, p = ps)
        return torch.tensor([[val]], device=device, dtype=torch.long), xxx[val]


In [ ]:
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    #transitions = memory.sample(BATCH_SIZE)

    transitions = memory.priority_sample(BATCH_SIZE)

    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    state_action_values = policy_net(state_batch).gather(1, action_batch)

    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1)[0].
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    # next_state_values = torch.zeros(BATCH_SIZE, device=device)
    # next_state_values[non_final_mask] = target_net(non_final_next_states).max(1)[0].detach()
    # # Compute the expected Q values
    expected_state_action_values = reward_batch #(next_state_values * GAMMA) + reward_batch

    # Compute Huber loss
    
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))
    #print(state_action_values - expected_state_action_values.unsqueeze(1))
    

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    for param in policy_net.parameters():
        param.grad.data.clamp_(-1, 1)
    optimizer.step()

In [ ]:
env.max_steps

In [ ]:
# l = [-0.1, -0.1,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1]

# np.sum([ x * np.power(0.8, i) for (i, x ) in  enumerate(l)])

# [(x.reward, x.expected_reward) for x in memory.priority_sample(3)]

In [ ]:
num_episodes = 10000
episode_durations = []
steps_done = 0
memory = ReplayMemory(2500)
policy_net.train()
target_net.eval()
sarsa_k = 1
window = 10

for i_episode in range(num_episodes):
    
    if i_episode % 50 == 0:
      print(i_episode)

    env.reset()
    state = get_screen().unsqueeze(0)

    
    temp_rewards = []
    temp_mempry =[]
    for t in count():
        action, expected_reward = select_action(state)
        _, reward, done, _ = env.step(action.item())
        
        next_state = get_screen().unsqueeze(0)

        if done:
          next_state = None
        temp_mempry.append((state, action, next_state, expected_reward))
        temp_rewards.append(reward)

        
        if done:

          for (index, data) in enumerate(temp_mempry):
            state, action, next_state, expected_reward = data 
            full_rewards = temp_rewards[index: index + window]

            if t == env.max_steps - 1:
              if len(full_rewards) < window:
                # print(index)
                continue
            # else:
            #   print('well {}'.format(t))

            reward = np.sum([ x * np.power(0.8, i) for (i, x ) in  enumerate(full_rewards)])
            reward = torch.tensor([reward], device=device).float()
            memory.push(state, action, next_state, reward, expected_reward)


        state = next_state

        if steps_done % BATCH_SIZE == 0:
          optimize_model()
        if done:
            episode_durations.append(t + 1)
            #plot_durations()
            break

    if i_episode % TARGET_UPDATE == 0:
        target_net.load_state_dict(policy_net.state_dict())
        target_net.eval()

print('Complete')
# env.render()
env.close()
plt.ioff()
plt.show()

In [ ]:
import pandas as pd 
durarion_counter = {}

# for duration in episode_durations:
#   if duration not in durarion_counter:
#     durarion_counter[duration] = 0
  
#   durarion_counter[duration] += 1


# durations_df = pd.DataFrame([(duration, counter) for (duration, counter) in durarion_counter.items()], columns =["duration", "counter"])

durations_df = pd.DataFrame([(iteration, duration) for (iteration, duration) in enumerate(episode_durations)], columns =["iteration", "duration"])

import seaborn as sns
sns.set_theme(style="whitegrid")
# ax = sns.barplot(x="duration", y="counter", data=durations_df)

sns.scatterplot(data=durations_df, x="iteration", y="duration")


In [ ]:
len(episode_durations)

In [ ]:
episode_durations[-50:]

In [ ]:
np.bincount(episode_durations, minlength=2)

In [ ]:
env.max_steps

In [ ]:
 EPS_END + (EPS_START - EPS_END) * math.exp(-1. * steps_done / EPS_DECAY)

In [ ]:
policy_net.eval()

In [ ]:
x  = policy_net(state)
# torch.argmax(x), x, target_net(state)

with torch.no_grad():
  xxx = policy_net(state).cpu().detach().numpy().flatten()
  m = xxx.min()
  vals = xxx - m + 0.0001

  s = np.sum(vals)
  ps = vals/s

np.max(ps), np.argmax(ps), ps

In [ ]:

x = env.reset()

In [ ]:
# p1, reward, done, p2 = env.step(action.item())
p2, done

In [ ]:
state = get_screen().unsqueeze(0)
i2 = state.squeeze(0).squeeze(0)
plt.gray()
plt.imshow(i2)
# plt.show()
# plt.imshow(i2)
# i2.shape

In [ ]:
# _, reward, done, _ = env.step(4)
# reward, done

In [ ]:
action = select_action(state)
_, reward, done, _ = env.step(action.item())
action.item(), reward

In [ ]:
next_state = get_screen()
i2 = state.squeeze(0).squeeze(0)
#plt.gray()
# plt.imshow(i2)
# plt.show()
plt.imshow(i2)
state = next_state.unsqueeze(0)

In [ ]:
_, reward, done, _ = env.step(action.item())
reward

In [ ]:
x = policy_net(state)
x, torch.argmax(x)

In [ ]:
x = policy_net(state)
xxx = x.cpu().detach().numpy().flatten()
m = xxx.min()

vals = xxx - m + 0.0001

s = np.sum(vals)
ps = vals/s


np.random.choice(n_actions, p = ps), np.max(ps), ps

In [ ]:
i2 = next_state.squeeze(0).squeeze(0) #.permute(1,2,0)
# plt.imshow(i2, cmap=plt.cm.gray)
plt.gray()
plt.imshow(i2)
plt.show()
state = next_state.unsqueeze(0)

In [ ]:

i2 = next_state.squeeze(0).permute(1,2,0)

plt.imshow(i2.numpy())

In [ ]:
state = next_state

In [ ]:
# BATCH_SIZE = 128
# GAMMA = 0.999
# EPS_START = 0.9
# EPS_END = 0.05
# EPS_DECAY = 10
# TARGET_UPDATE = 10
# import math 
# steps_done =1
# eps_threshold = EPS_END + (EPS_START - EPS_END) * math.exp(-1. * steps_done / EPS_DECAY)

# eps_threshold

In [ ]:
(EPS_START - EPS_END) * math.exp(-1. * 0 / EPS_DECAY)

In [ ]:
(EPS_START - EPS_END) * math.exp(-1. * steps_done / EPS_DECAY)

In [ ]:
eps_to_actions[3739][0:30]

In [ ]:
eps_to_actions[0][0:20]

In [ ]:
_, reward, done, _ = env.step(action.item())
reward

In [ ]:
torch.tensor([reward], device=device).detach().cpu().numpy().dtype